In [135]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [136]:
N = 150 # Total no of points
D = 5  # Dimension of space
K = 3 # Number of clusters

In [137]:
from sklearn.datasets import load_iris
iris_data = load_iris()
X=iris_data.data

In [138]:
from sklearn.metrics import pairwise_distances 
from scipy.spatial import distance 
import math
X=iris_data.data

a=np.zeros(X.shape[1])
for i in range(0,X.shape[1]):
    a[i]=np.dot(X.T[i],np.ones(X.shape[0]))/X.shape[0]
a=np.matrix(a)
dists = distance.cdist(X, a, 'euclidean')
R=np.max(dists)
Xf=np.zeros((150,4))
Xe=np.zeros((150))
for i in range(0,150):
    Xe[i]=R*((np.dot(X[i]**2,np.ones(4))-math.pow(R,2))/((np.dot(X[i]**2,np.ones(4)))+math.pow(R,2)))
    Xf[i]=R*(2*R/((np.dot(X[i]**2,np.ones(4)))+math.pow(R,2))*X[i])
    
X=np.column_stack((Xf, Xe)) 
A=np.dot(X,X.T)/(R**2)       
for i in range(0,X.shape[0]):
        A[i][i]=int(1)
for i in range(0,X.shape[0]):
    for j in range(0,X.shape[0]):
        if A[i,j]>1:
            A[i][j]=int(1)          
A=R*np.arccos(A)
input_data = X.T

In [139]:
def zij(A,i,j,lam,N):
    if i==j:
        return 0.0
    else:
        numerator = 2 * np.exp(-(np.square(A[i,j]))/lam)
        #print(numerator)
        sum_i=0
        sum_j=0
        for h in range(N):
            if h!=i:
                sum_i += np.exp(-(np.square(A[i,h]))/lam)
        for h in range(N):
            if h!=j:
                sum_j += np.exp(-(np.square(A[j,h]))/lam)
        return numerator/(sum_i+sum_j)

In [140]:
Z =np.zeros((N,N), dtype='float64')
for i in range(N):
    for j in range(N):
        Z[i,j] = zij(A,i,j,0.14094,N)

In [141]:
# Check sparsity by counting the number of zeros
cz = 0
for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        if Z[i,j] < 1e-5 and Z[i,j] > -1e-5:
            cz += 1
print(cz)

10424


In [142]:
from scipy import linalg
LN = np.subtract(np.eye(N,N),Z)
eigenvals, eigenvcts = linalg.eig(LN)
eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)
eig = eigenvals.reshape((N,1))


In [143]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [144]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

1
0
2


In [145]:
indices = np.asarray(indices)

In [146]:
zero_eigenvals_index = np.array(indices)

In [147]:
eigenvals[zero_eigenvals_index]

array([[0.01272122],
       [0.01434284],
       [0.36476886]])

In [148]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2
0,-0.001818,0.165344,-0.000042
1,-0.001601,0.145674,-0.000043
2,-0.001631,0.148509,-0.000054
3,-0.001535,0.139676,-0.000047
4,-0.001787,0.162512,-0.000043


In [149]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K)

In [150]:
print(cluster)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 1 1 2 1 2 1
 1 1 2 2 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [151]:
c0 = 0
for l in cluster:
    if l == 0:
        c0 += 1
print('c0=',c0)


c1 = 0
for l in cluster:
    if l == 1:
        c1 += 1
print('c1=',c1)


c2 = 0
for l in cluster:
    if l == 2:
        c2 += 1
print('c2=',c2)

c0= 50
c1= 38
c2= 62


In [152]:
pred = np.asarray(cluster)
orig = iris_data.target
from sklearn.metrics import normalized_mutual_info_score
print("NMI = " + str(normalized_mutual_info_score(orig, pred)))

NMI = 0.7856651817981657
